In [ ]:
using Pkg

Pkg.activate()

inlcude("generate_ivt_fields.jl")

using preprocessing

In [ ]:
"""This function loads data in given geographic bounds. It supports loading values going 'over the end' like the lon rage from 270-40 NOTE: It is expected that the longitude is given in values from 0-360 deg and lat in range from -90:90""" 
function load_data_in_geo_bounds_typed(dataset_path, field_id::Union{String, Symbol, Missing}, geo_bnds::GeographicBounds, indices...)::Array

  lon_normal_range = geo_bnds.lon_bounds[1] < geo_bnds.lon_bounds[2]
  lat_normal_range = geo_bnds.lat_bounds[1] < geo_bnds.lat_bounds[2]

  # first load the file information

  

  result_dim = length(indices) + 2


  if lon_normal_range & lat_normal_range
    return dataset[field_id][geo_bnds.lon_indices[1]:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:geo_bnds.lat_indices[2], indices...]
  elseif !lon_normal_range & lat_normal_range
    
    lon_first = dataset[field_id][geo_bnds.lon_indices[1]:end, geo_bnds.lat_indices[1]:geo_bnds.lat_indices[2],indices...]
    lon_second = dataset[field_id][1:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:geo_bnds.lat_indices[2],indices...]
    
    return vcat(lon_first, lon_second)
  elseif lon_normal_range & !lat_normal_range
    
    lat_first = dataset[field_id][geo_bnds.lon_indices[1]:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:end,indices...]
    lat_second = dataset[field_id][geo_bnds.lon_indices[1]:geo_bnds.lon_indices[2], 1:geo_bnds.lat_indices[2],indices...]
    
    return hcat(lat_first, lat_second)
  else
    # last case is both are over 
    lon_f_lat_f = dataset[field_id][geo_bnds.lon_indices[1]:end, geo_bnds.lat_indices[1]:end,indices...]
    lon_f_lat_s = dataset[field_id][geo_bnds.lon_indices[1]:end, 1:geo_bnds.lat_indices[2],indices...]

    lon_s_lat_f = dataset[field_id][1:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:end,indices...]
    lon_s_lat_s = dataset[field_id][1:geo_bnds.lon_indices[2], 1:geo_bnds.lat_indices[2],indices...]

    return vcat(hcat(lon_f_lat_f, lon_f_lat_s), hcat(lon_s_lat_f, lon_s_lat_s))
  end
end

In [1]:
using NCDatasets
using DataStructures

function create_benchmark_ds(path, varname, data)
    
    vertical_size = size(data, 3)

    ps = rand(size(data, 1), size(data, 2), size(data, 4))

    ap = rand(vertical_size)

    b = rand(vertical_size)

    NCDataset(path,"c",attrib = OrderedDict("title" => "this is a test file")) do ds
        # Define the variable temperature. The dimension "lon" and "lat" with the
        # size 100 and 110 resp are implicitly created
        defVar(ds,varname,data,("lon","lat", "lev", "time"))
        defVar(ds,"ps",ps,("lon","lat", "time"))
        defVar(ds,"ap",ap,("lev",))
        defVar(ds,"b",b,("lev",))
    end
    
end

create_benchmark_ds (generic function with 1 method)

In [2]:
data = [Float32(l+i+j/k) for i = 1:60, j = 1:50, k = 1:42, l = 1:5000]

println("Data size in mem: $(sizeof(data)/1000000) MB")
for id in ["hus", "ua", "va"]
    path = "sample_data/benchmark_$id.nc"
    create_benchmark_ds(path, id, data)
end



Data size in mem: 2520.0


In [1]:
using BenchmarkTools
using NCDatasets


bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][:, :, :, :]
    data_ua = ds[:ua][:, :, :, :]
    data_va = ds[:va][:, :, :, :]
    data_ps = ds[:ps][:, :, :]

end

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  4.199 s …    4.380 s  ┊ GC (min … max): 13.76% … 17.42%
 Time  (median):     4.289 s               ┊ GC (median):    15.63%
 Time  (mean ± σ):   4.289 s ± 128.221 ms  ┊ GC (mean ± σ):  15.63% ±  2.59%

  █                                                        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.2 s          Histogram: frequency by time         4.38 s <

 Memory estimate: 7.15 GiB, allocs estimate: 3452.

In [2]:
bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][1:40, 1:20, :, :]
    data_ua = ds[:ua][1:40, 1:20, :, :]
    data_va = ds[:va][1:40, 1:20, :, :]
    data_ps = ds[:ps][1:40, 1:20, :]

end samples = 15

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.710 s …    1.932 s  ┊ GC (min … max):  6.43% … 17.18%
 Time  (median):     1.753 s               ┊ GC (median):     9.31%
 Time  (mean ± σ):   1.799 s ± 117.734 ms  ┊ GC (mean ± σ):  11.22% ±  5.56%

  █          █                                             █  
  █▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.71 s         Histogram: frequency by time         1.93 s <

 Memory estimate: 1.91 GiB, allocs estimate: 3452.

In [15]:
bmable = @benchmarkable  NCDataset(["sample_data/sample_hus_dataset_200_timesteps.nc", "sample_data/sample_ua_dataset_200_timesteps.nc", "sample_data/sample_va_dataset_200_timesteps.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][:, :, :, :]::Array{Float32, 4}
    data_ua = ds[:ua][:, :, :, :]::Array{Float32, 4}
    data_va = ds[:va][:, :, :, :]::Array{Float32, 4}
    data_ps = ds[:ps][:, :, :]::Array{Float64, 3}

end

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.155 s …    1.416 s  ┊ GC (min … max): 0.09% … 18.81%
 Time  (median):     1.311 s               ┊ GC (median):    7.74%
 Time  (mean ± σ):   1.298 s ± 119.586 ms  ┊ GC (mean ± σ):  9.06% ±  9.76%

  █                   █                           █        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█ ▁
  1.15 s         Histogram: frequency by time         1.42 s <

 Memory estimate: 2.35 GiB, allocs estimate: 2999.

In [18]:
bmable = @benchmarkable  NCDataset(["sample_data/sample_hus_dataset_200_timesteps.nc", "sample_data/sample_ua_dataset_200_timesteps.nc", "sample_data/sample_va_dataset_200_timesteps.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][1:80, 1:50, :, :]::Array{Union{Missing, Float32}, 4}
    data_ua = ds[:ua][1:80, 1:50, :, :]::Array{Union{Missing, Float32}, 4}
    data_va = ds[:va][1:80, 1:50, :, :]::Array{Union{Missing, Float32}, 4}
    data_ps = ds[:ps][1:80, 1:50, :]::Array{Float32, 3}

end

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  1.172 s …   1.359 s  ┊ GC (min … max):  8.10% … 20.96%
 Time  (median):     1.197 s              ┊ GC (median):     7.94%
 Time  (mean ± σ):   1.222 s ± 77.638 ms  ┊ GC (mean ± σ):  10.84% ±  5.82%

  ██     █ █                                              █  
  ██▁▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.17 s         Histogram: frequency by time        1.36 s <

 Memory estimate: 972.12 MiB, allocs estimate: 12839.

In [2]:
using Pkg

Pkg.activate(".")


  Activating project at `~/Documents/Uni/Master/MA/preprocessing`


In [6]:
include("generate_ivt_fields.jl")

using NCDatasets
using .preprocessing
using BenchmarkTools


bmable = @benchmarkable NCDataset(["sample_data/sample_hus_dataset_200_timesteps.nc", "sample_data/sample_ua_dataset_200_timesteps.nc", "sample_data/sample_va_dataset_200_timesteps.nc"]; aggdim = "") do dataset
    println("Threads available: $(Threads.nthreads())")
    
    hus_data = dataset[:hus][:, :, :, :]
    ua_data = dataset[:ua][:, :, :, :]
    va_data = dataset[:va][:, :, :, :]
    ps_data = dataset[:ps][:, :, :]
    
    lon_size = size(hus_data, 1)
    lat_size = size(hus_data, 2)

    # these variables are used for calculation of pressure levels at each specific lat, lon, time coordinate: p = ap + b * ps
    ap = dataset[:ap][:]
    b = dataset[:b][:]
    time_size = size(dataset[:time], 1)
      
end
tune!(bmable)

run(bmable)

Threads available: 8
Threads available: 8
Threads available: 8
Threads available: 8
Threads available: 8
Threads available: 8
Threads available: 8


BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  2.375 s …    2.617 s  ┊ GC (min … max):  6.51% … 14.81%
 Time  (median):     2.504 s               ┊ GC (median):    10.21%
 Time  (mean ± σ):   2.499 s ± 121.308 ms  ┊ GC (mean ± σ):  10.65% ±  4.16%

  █                             █                          █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.37 s         Histogram: frequency by time         2.62 s <

 Memory estimate: 4.37 GiB, allocs estimate: 13545.

In [ ]:
using NCDatasets

function load_data_to_var!(path, variable_name, var, indices...)
    
    NCDataset(path) do ds
        NCDatasets.load!(variable(ds, variable_name),var, indices...)
    end
end

load_data_to_var! (generic function with 1 method)

In [3]:
using NCDatasets

function load_data_fast(path, variable_name, var, indices...)
    
    NCDataset(path) do ds
        NCDatasets.load!(variable(ds, variable_name),var, indices...)
    end
end

load_data_to_var! (generic function with 1 method)

In [5]:
using BenchmarkTools
@benchmark begin
    hus_data = zeros(Float32, 60, 50, 42, 5000)
    # hus_data_sub = hus_data[:, :, :, 1:200]
    load_data_to_var!("sample_data/benchmark_hus.nc", "hus", hus_data, :, :, :, :)
end

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.370 s …    1.632 s  ┊ GC (min … max):  0.04% … 16.32%
 Time  (median):     1.594 s               ┊ GC (median):    12.39%
 Time  (mean ± σ):   1.547 s ± 122.542 ms  ┊ GC (mean ± σ):  10.70% ±  7.08%

  █                                         █             ██  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁██ ▁
  1.37 s         Histogram: frequency by time         1.63 s <

 Memory estimate: 2.35 GiB, allocs estimate: 61.

In [14]:
@benchmark begin
    NCDataset("sample_data/sample_hus_dataset_200_timesteps.nc") do ds
        hus_data = ds["hus"][:, :, :, :]::Array{Union{Missing, Float32}, 4}
    end
end

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  969.426 ms …   1.142 s  ┊ GC (min … max):  0.19% … 16.25%
 Time  (median):        1.095 s              ┊ GC (median):    12.38%
 Time  (mean ± σ):      1.086 s ± 69.798 ms  ┊ GC (mean ± σ):  11.47% ±  6.40%

  █                                       █  █              ██  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁██ ▁
  969 ms          Histogram: frequency by time          1.14 s <

 Memory estimate: 1.45 GiB, allocs estimate: 195.

In [2]:
using NetCDF
using BenchmarkTools

@benchmark begin
    hus_data = ncread("sample_data/benchmark_hus.nc", "hus")
end

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.177 s …    1.413 s  ┊ GC (min … max): 0.04% … 17.04%
 Time  (median):     1.273 s               ┊ GC (median):    7.52%
 Time  (mean ± σ):   1.284 s ± 120.144 ms  ┊ GC (mean ± σ):  8.42% ±  9.03%

  █ █                                         █            █  
  █▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.18 s         Histogram: frequency by time         1.41 s <

 Memory estimate: 2.35 GiB, allocs estimate: 137.

In [20]:
using NetCDF

@benchmark begin
    hus_data_subset = ncread("sample_data/sample_hus_dataset_200_timesteps.nc", "hus", start=[1, 1, 1, 1], count=[80, 50, -1, -1])
end

BenchmarkTools.Trial: 20 samples with 1 evaluation.
 Range (min … max):  228.324 ms … 361.635 ms  ┊ GC (min … max): 0.00% … 36.25%
 Time  (median):     240.455 ms               ┊ GC (median):    0.83%
 Time  (mean ± σ):   253.533 ms ±  34.847 ms  ┊ GC (mean ± σ):  8.58% ± 10.44%

  █                                                              
  ██▅▅▅▁▅▁▁▁▁██▅▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  228 ms           Histogram: frequency by time          362 ms <

 Memory estimate: 143.51 MiB, allocs estimate: 992.

In [8]:
# Loading data and setting it all up

using NetCDF


hus_data = ncread("sample_data/sample_hus_dataset_200_timesteps.nc", "hus")
ua_data = ncread("sample_data/sample_ua_dataset_200_timesteps.nc", "ua")
va_data = ncread("sample_data/sample_va_dataset_200_timesteps.nc", "va")
ps_data = ncread("sample_data/sample_va_dataset_200_timesteps.nc", "ps")

lon_size = size(hus_data, 1)
lat_size = size(hus_data, 2)

# these variables are used for calculation of pressure levels at each specific lat, lon, time coordinate: p = ap + b * ps
ap = ncread("sample_data/sample_hus_dataset_200_timesteps.nc", "ap")
b = ncread("sample_data/sample_hus_dataset_200_timesteps.nc", "b")
time_vals = ncread("sample_data/sample_hus_dataset_200_timesteps.nc", "time")
time_size = length(time_vals)

200

In [11]:
include("IVT.jl")

using .IVT


println("Time used for calculating the IVT field: ")
@benchmark begin

    result_data::Array{Union{Float64, Missing}, 3} = zeros(lon_size, lat_size, time_size)

    
    Threads.@threads for time in 1:time_size
    for lat in 1:lat_size
        for lon in 1:lon_size
        
        hus_column::Vector{Union{Float32, Missing}} = hus_data[lon, lat, :, time]
        ua_column::Vector{Union{Float32, Missing}} = ua_data[lon, lat, :, time]
        va_column::Vector{Union{Float32, Missing}} = va_data[lon, lat, :, time]

        ps = ps_data[lon, lat, time]
        pressure_levels = ap + b * ps

        vertical_column_data = VerticalColumnData(hus_column, ua_column, va_column, pressure_levels, ps)

        result_data[lon, lat, time] = ivt_of_column(vertical_column_data)

        end
    end
    end
    
end

Time used for calculating the IVT field: 


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 25.624 s (65.19% GC) to evaluate,
 with a memory estimate of 53.67 GiB, over 2845939487 allocations.

In [17]:
include("IVT.jl")

using .IVT


println("Time used for calculating the IVT field: ")
@benchmark begin

    result_data::Array{Union{Float64, Missing}, 3} = zeros(lon_size, lat_size, time_size)

    
    Threads.@threads for time in 1:time_size
    for lat in 1:lat_size
        for lon in 1:lon_size
        
            ps = ps_data[lon, lat, time]
            pressure_levels = ap + b * ps

            result_data[lon, lat, time] = IVT.ivt_of_column_vectors(ps, pressure_levels, hus_data[lon, lat, :, time], ua_data[lon, lat, :, time], va_data[lon, lat, :, time])

        end
    end
    end
    
end

Time used for calculating the IVT field: 


BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.250 s … 5.864 s  ┊ GC (min … max):  0.00% … 79.44%
 Time  (median):     1.320 s            ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.812 s ± 2.644 s  ┊ GC (mean ± σ):  55.23% ± 45.86%

  █                                                     ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.25 s        Histogram: frequency by time       5.86 s <

 Memory estimate: 6.43 GiB, allocs estimate: 47961888.